In [1]:
import scipy.io
import numpy as np
import scipy.io as spio

def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    data = spio.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

def _check_keys(dict):
    '''
    checks if entries in dictionary are mat-objects. If yes
    todict is called to change them to nested dictionaries
    '''
    for key in dict:
        if isinstance(dict[key], spio.matlab.mio5_params.mat_struct):
            dict[key] = _todict(dict[key])
    return dict        

def _todict(matobj):
    '''
    A recursive function which constructs from matobjects nested dictionaries
    '''
    dict = {}
    for strg in matobj._fieldnames:
        elem = matobj.__dict__[strg]
        if isinstance(elem, spio.matlab.mio5_params.mat_struct):
            dict[strg] = _todict(elem)
        else:
            dict[strg] = elem
    return dict

In [2]:
mat_reverse = loadmat('/media/yipeng/data/movie_2021/Movie_Analysis/final_result_outputs_reverse/LSTM/figure4a.mat')
mat_fake = loadmat('/media/yipeng/data/movie_2021/Movie_Analysis/final_result_outputs_fake_label/LSTM/figure4a.mat')
mat_old = loadmat('/media/yipeng/data/movie_2021/Movie_Analysis/final_result_outputs_old_feb/LSTM/figure4a.mat')
patient_names = ['p_431', 'p_435', 'p_436', 'p_441']

In [3]:
label = np.load("/media/yipeng/data/movie_2021/Movie_Analysis/training_data_1_zero/433/label.npy")
label.shape

(18829, 4, 3)

In [4]:
res = []
for i in range(4):
    res.append(label[:,i,0].sum())

In [5]:
prob = np.array(res)/len(label)

In [6]:
prob

array([0.17919167, 0.10733443, 0.10016464, 0.08683414])

In [7]:
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from sklearn.metrics import recall_score
def check_activations(mat, prob):
    res = []
    for p in patient_names:
        character = mat[p]["character_each_clip"]
        print(p)
        for i in range(4):
            nums = np.random.choice([1, 0], size=len(character), p=[prob[i], 1-prob[i]])
            f1 = f1_score(character[:,i], nums)
            recall_score = f1_score(character[:,i], nums)
            accuracy = np.sum(nums == character[:,i])/len(character)
            print(accuracy)

In [8]:
check_activations(mat_old, prob)


p_431
0.6066666666666667
0.8066666666666666
0.7866666666666666
0.8
p_435
0.65
0.8166666666666667
0.7866666666666666
0.8066666666666666
p_436
0.6466666666666666
0.8133333333333334
0.7933333333333333
0.7933333333333333
p_441
0.6840277777777778
0.8055555555555556
0.8229166666666666
0.8125


In [9]:
n, p = 10, .5  # number of trials, probability of each trial
s = np.random.binomial(n, p, 1000)

In [10]:
len(s)

1000